In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation, BatchNormalization as BatchNorm
from tensorflow.keras.callbacks import ModelCheckpoint

/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alexm/.conda/envs/tf-gpu/lib/python3.6/site-packages

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()  # True/False

# Only check for GPUs with CUDA support
tf.test.is_gpu_available(cuda_only=True)

True

In [3]:
# Load in original and preprocessed MIDI data
np_notes = np.load("cached_data/notes.npy")
np_network_input = np.load("cached_data/network_input.npy")
np_network_output = np.load("cached_data/network_output.npy")

In [4]:
# Ascertain the number of vocabulary available to our network for training
n_vocab = len(set(np_notes))

In [5]:
# Instantiate a nine-layer sequential model with Keras
model = Sequential()

model.add(LSTM(
    512,
    input_shape=(np_network_input.shape[1], np_network_input.shape[2]),
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Load the weights to each node
model.load_weights('model/weights.h5')

In [ ]:
# Establish a checkpoint file and callback for training
filepath = "model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

callbacks_list = [checkpoint]
model.fit(np_network_input, np_network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
73198/73198 [==============================] - 562s 8ms/step - loss: 4.6153
Epoch 2/200
73198/73198 [==============================] - 500s 7ms/step - loss: 4.5714
Epoch 3/200
 7808/73198 [==>...........................] - ETA: 7:27 - loss: 4.5462